 In this exercise, we will implement an encoder-decoder algorithm that allows for translation. Good luck!

In [ ]:
## 1 - TODO : Import and load the french and english sentence (2 .txt files)

##You will store the English sentences in a variable named 'english_sentences' and the French ones in a variable named 'french_sentences'.

In [1]:
with open('data_english.txt', 'r') as file:
    english_sentences = file.readlines()

with open('data_french.txt', 'r') as file:
    french_sentences = file.readlines()

In [2]:
## 2 - TODO : HOW MANY SENTENCES DO YOU HAVE ?

In [3]:
len(french_sentences), len(english_sentences)

(137860, 137860)

In [4]:
## 3 - TODO : Display the first 5 sentences and their translation.
french_sentences[:5], english_sentences[:5]

(["new jersey est parfois calme pendant l' automne , et il est neigeux en avril .\n",
  'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .\n',
  'california est généralement calme en mars , et il est généralement chaud en juin .\n',
  'les états-unis est parfois légère en juin , et il fait froid en septembre .\n',
  'votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .\n'],
 ['new jersey is sometimes quiet during autumn , and it is snowy in april .\n',
  'the united states is usually chilly during july , and it is usually freezing in november .\n',
  'california is usually quiet during march , and it is usually hot in june .\n',
  'the united states is sometimes mild during june , and it is cold in september .\n',
  'your least liked fruit is the grape , but my least liked is the apple .\n'])

In [5]:
## TODO : 4 - Count number of words in all english sentences
sum_french = sum(len(elt) for elt in french_sentences)
sum_french

9847063

In [6]:
##  4-2 - Count number of different unique words in all french sentences
unique_words_fr = set()
for sentence in french_sentences:
    unique_words_fr.update(set(sentence.lower().split()))
len(unique_words_fr)

354

In [7]:
##  4-3 - Display the 10 most frequent french word in the sentences
from collections import Counter
all_words_fr = []
for sentence in french_sentences:
    all_words_fr.extend(sentence.lower().split())
word_counts_fr = Counter(all_words_fr)
for word, count in word_counts_fr.most_common(10):
    print(f"{word}: {count} times")

est: 196809 times
.: 135619 times
,: 123135 times
en: 105768 times
il: 84079 times
les: 65255 times
mais: 63987 times
et: 59851 times
la: 49861 times
parfois: 37746 times


In [8]:
## TODO  5 - Do the same questions with english sentences
## TODO : 5-1 - Count number of words in all english sentences
sum_english = sum(len(elt) for elt in english_sentences)
sum_english

9085266

In [9]:
##  TODO 5-2 - Count number of different unique words in all english sentences
unique_words_en = set()
for sentence in english_sentences:
    unique_words_en.update(set(sentence.lower().split()))
len(unique_words_en)

227

In [10]:
##  5-3 - Display the 10 most frequent english word in the sentences
all_words_en = []
for sentence in english_sentences:
    all_words_en.extend(sentence.lower().split())
word_counts_en = Counter(all_words_en)
for word, count in word_counts_en.most_common(10):
    print(f"{word}: {count} times")

is: 205858 times
,: 140897 times
.: 129039 times
in: 75525 times
it: 75137 times
during: 74933 times
the: 67628 times
but: 63987 times
and: 59850 times
sometimes: 37746 times


In [5]:
## TODO 6 - Create a function (named tokenize) that allows you to tokenize your sentences:
## The function return list of tokenized sentences and the tokenizer
##Indeed, the network only takes sequences of numbers!
from tensorflow.keras.preprocessing.text import Tokenizer
def tokenize(text) :
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text)
    text_token = tokenizer.texts_to_sequences(text)
    return text_token, tokenizer

2023-11-21 16:17:03.411332: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
#tokenize(french_sentences[:2])

In [13]:
#tokenize(english_sentences[:2])

In [6]:
## 7 - Create a function (named pad) that allows you to pad your data. It will take two arguments: 'x,' which is a list of sequences, and 'length,' which is the maximum length we desire. 
##By default, 'length' will be set to None, meaning padding with the max of sequence length
## PS : you can use the sequence.pad_sequences function
from tensorflow.keras.preprocessing import sequence
def pad(x, length=None):
    pad_data = sequence.pad_sequences(x, maxlen = length,padding = 'post')
    return pad_data


In [ ]:
pad()

In [7]:
## 8-  Use this function (and understand it) and apply on french and english sentence

def preprocess(x,y):
    
    """"
     x : list de phrases à preprocesser
     y : list de phrases à preprocesser
     return : np array de phrases preprocesser , et leur tokenizer
    """
    
    preprocess_x , x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    preprocess_x = pad(preprocess_x,length = None)
    preprocess_y = pad(preprocess_y,length = None)
    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    
    return preprocess_x,preprocess_y,x_tk,y_tk

preproc_english, preproc_french , english_tokenizer , french_tokenizer =  preprocess(english_sentences,french_sentences)

In [23]:
#preprocess(english_sentences,french_sentences)[:2]

In [46]:
len(english_tokenizer.index_word)

199

In [8]:
## 9 - Apply this function 

import numpy as np
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [21]:
## 11 - We are going to translate english sentences to french sentences 
### Create a Sequential model with : * Embedding layer (input_dim = number of word in english dictionnary,
#                                                       output_dim = 256 (embedding size),
#                                                       input_length = size max of the sequence (21))
#.                                   * One LSTM Layer with 256 neurons (dont forget return_sequence = True)
#                                    * Dropout(a) with a = 0.5
#                                    * Dense(b) with b = french_vocab_size (number of words in french_vocab_size)

In [9]:
## 12 - Reshape the input with this line 
tmp_x = pad(preproc_english, preproc_french.shape[1])
tmp_x

array([[17, 23,  1, ...,  0,  0,  0],
       [ 5, 20, 21, ...,  0,  0,  0],
       [22,  1,  9, ...,  0,  0,  0],
       ...,
       [19,  1, 10, ...,  0,  0,  0],
       [24,  1, 10, ...,  0,  0,  0],
       [ 5, 84,  1, ...,  0,  0,  0]], dtype=int32)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping

In [25]:
def model_LSTM():
    model = Sequential()
    model.add(Embedding(input_dim = len(english_tokenizer.index_word) + 1, output_dim = 256 , input_length = 21))
    model.add(LSTM(units = 256, return_sequences = True))
    model.add(Dropout(0.5))
    model.add(Dense(units = len(french_tokenizer.index_word) + 1, activation = 'softmax'))
    return model

In [26]:
##13 - Do model.summary() and find the number of estimated paramters using your own resources
model = model_LSTM()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 21, 256)           51200     
                                                                 
 lstm_1 (LSTM)               (None, 21, 256)           525312    
                                                                 
 dropout_1 (Dropout)         (None, 21, 256)           0         
                                                                 
 dense_1 (Dense)             (None, 21, 345)           88665     
                                                                 
Total params: 665177 (2.54 MB)
Trainable params: 665177 (2.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
(256+256)*256*4 +(256*4)

525312

In [27]:
## 14 - Apply your model and print the original sentence, the result of the model and the desired output (true translation)
from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(), loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])
model.fit(tmp_x, preproc_french, validation_split = 0.2, batch_size = 32, epochs = 2)

Epoch 1/2
3447/3447 [==============================] - 590s 170ms/step - loss: 0.7268 - accuracy: 0.8101 - val_loss: 0.2910 - val_accuracy: 0.9075
Epoch 2/2
3447/3447 [==============================] - 612s 177ms/step - loss: 0.3031 - accuracy: 0.9057 - val_loss: 0.2357 - val_accuracy: 0.9223


In [ ]:
#loss, accuracy = model.evaluate(tmp_x, preproc_french, verbose = 0)
#loss_accuracy

In [36]:
(tmp_x[:1])[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0, 17, 23,  1,  8, 67,  4, 39,  7,  3,
        1, 55,  2, 44], dtype=int32)

In [37]:
len((tmp_x[:1])[0])

21

In [39]:
len(english_sentences[0])

73

In [40]:
len(french_sentences[0])

79

In [28]:
print("Prediction:")
print(logits_to_text(model.predict(tmp_x[:1])[0],french_tokenizer))
print("\nCorrect Translation:")
print(french_sentences[0])
print("\nOriginal text:")
print(english_sentences[0])

Prediction:
1/1 [==============================] - 1s 803ms/step
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


Original text:
new jersey is sometimes quiet during autumn , and it is snowy in april .



In [29]:
print("Prediction:")
print(logits_to_text(model.predict(tmp_x[3:4])[0], french_tokenizer))
print("\nCorrect Translation:")
print(french_sentences[3])
print("\nOriginal text:")
print(english_sentences[3])

Prediction:
1/1 [==============================] - 0s 54ms/step
les états unis est parfois doux en juin et il est froid en septembre <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
les états-unis est parfois légère en juin , et il fait froid en septembre .


Original text:
the united states is sometimes mild during june , and it is cold in september .

